# Co-Win Web API's

In [ ]:
import requests

## 1. Find the ID of our district

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

In [ ]:
response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states", headers=headers)
print(response.status_code)

In [ ]:
response.json()

Say we want the districts in `tamil nadu`, we will get the state's code from the list above. Its value is `31`

In [ ]:
# State code for tamil nadu is 31

# Now we get the districts here
response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/31", headers=headers)
print(response.status_code)

In [ ]:
response.json()

We note down the code for a district, say the code for `madurai` is `540`

## 2. Decide the date to be queried

In [ ]:
import datetime
import pytz

In [ ]:
# Get the current time
now = datetime.datetime.now()

# Set the timezone
tz = pytz.timezone('Asia/Kolkata')

# Convert the current time into this timezone
now = now.astimezone(tz)

now

In [ ]:
# Get this into the required format

current_date = now.strftime("%d-%m-%Y")
current_date

In [ ]:
# To get the hour

now.hour

In [ ]:
# Increase the day to the next day
next_date = now + datetime.timedelta(days=1)
next_date

## 2. Get the vaccine slots based on this district code

In [ ]:
# Now we get the vaccine slots here

parameters = {
    'district_id' : "540",
    "date" : current_date,
}
response = requests.get("https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByDistrict",                              headers=headers, params=parameters)
print(response.status_code)

In [ ]:
response.json()

## 3. Parse through this data

In [ ]:
# Save this json response
data = response.json()

candidate_centers = []
# Parse through this json data and save the ones which have availability
for center in data['sessions'] :
    if (
        (center['vaccine'] in ['COVISHIELD', 'SPUTNIK V']) and
        (center['min_age_limit'] >= 18 ) and 
        (center['available_capacity_dose2'] > 0 )
    ) :
        cur_candidate = {
            'name' : center['name'],
            'num_dose_1' : center['available_capacity_dose1'],
            'fee' : center['fee'],
            'vaccine' : center['vaccine'],
            'min_age' : center['min_age_limit'],
            'slots' : center['slots']
        }
        candidate_centers.append(cur_candidate)

In [ ]:
candidate_centers

## 4. Generate the text version of these centers

In [ ]:
def generateText(candidate_centers) :
    """
    Given a list of candidate centers, converts these dictionary content to a text format
    """

    complete_text = ""

    for i, center in enumerate(candidate_centers) :
        complete_text += f"{i+1}<b><u>{center['name']} :</u></b>"
        complete_text += f"Doses Available : {center['num_dose_1']}"
        complete_text += f"Fee : {center['fee']}"
        complete_text += f"Vaccine : {center['vaccine']}"
        complete_text += f"Age group : {center['min_age_limit']}+ years"
        complete_text += f"Slots : {center['slots'][0]}"
        for j in range(1, len(center['slots'])) :
            complete_text += f"{center['slots'][j]}"
        complete_text += "---------------------"